# 🚀 Fine-Tune LLMs with Together AI

This notebook guides you through fine-tuning a Large Language Model using Together AI.

**Why Together AI?**
- Wide selection of open-source models (Llama, Mistral, Qwen, etc.)
- Competitive pricing for fine-tuning
- Good documentation and community
- Serverless deployment after fine-tuning

**What you'll learn:**
1. Prepare and validate your training dataset
2. Upload to Together AI
3. Configure and launch fine-tuning
4. Deploy and test your model

**Prerequisites:**
- Together AI account (https://together.ai)
- API key from Together

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install together pandas jsonlines

In [ ]:
import os
import json
import pandas as pd
import together
from together import Together

# Set your Together API key
try:
    from google.colab import userdata
    TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')
except:
    TOGETHER_API_KEY = input("Enter your Together API key: ")

os.environ['TOGETHER_API_KEY'] = TOGETHER_API_KEY
client = Together(api_key=TOGETHER_API_KEY)

print("✅ Together AI client configured")

## 2. Dataset Preparation

Together AI expects data in **JSONL format** with conversation-style messages.

### Format:
```json
{"messages": [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

### Dataset Requirements:
- Minimum 10 examples (recommended: 50-1000+)
- Each example should be a complete conversation
- Consistent formatting across examples

In [ ]:
# ============================================================
# Sample Dataset: SQL Query Assistant
# Replace with your own domain-specific data!
# ============================================================

SYSTEM_PROMPT = """You are an expert SQL assistant. You help users write efficient, correct SQL queries.
When writing queries:
- Use clear table aliases
- Add comments explaining complex logic
- Suggest indexes when relevant
- Follow best practices for the specific database type"""

sample_data = [
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "Write a query to find the top 10 customers by total order value"},
            {"role": "assistant", "content": """Here's a query to find the top 10 customers by total order value:

```sql
SELECT 
    c.customer_id,
    c.customer_name,
    c.email,
    SUM(o.total_amount) AS total_order_value,
    COUNT(o.order_id) AS order_count
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'completed'  -- Only count completed orders
GROUP BY c.customer_id, c.customer_name, c.email
ORDER BY total_order_value DESC
LIMIT 10;
```

**Performance tip:** Ensure you have indexes on:
- `orders.customer_id` (for the join)
- `orders.status` (for filtering)

Would you like me to modify this for a specific time period or add additional filters?"""}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "How do I find duplicate records in a table?"},
            {"role": "assistant", "content": """Here are several approaches to find duplicate records:

**Method 1: Using GROUP BY and HAVING**
```sql
-- Find duplicate emails
SELECT email, COUNT(*) as duplicate_count
FROM users
GROUP BY email
HAVING COUNT(*) > 1;
```

**Method 2: Get all rows with duplicates**
```sql
SELECT *
FROM users u1
WHERE EXISTS (
    SELECT 1 FROM users u2
    WHERE u2.email = u1.email
    AND u2.id != u1.id
);
```

**Method 3: Using window functions (keeps all columns)**
```sql
WITH duplicates AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY email ORDER BY created_at) as rn
    FROM users
)
SELECT * FROM duplicates WHERE rn > 1;  -- Shows duplicate rows
```

Which method works best depends on:
- Whether you need to see all duplicate rows or just counts
- Your database's support for window functions
- Table size (window functions can be slower on large tables)

What specific columns do you want to check for duplicates?"""}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "Explain the difference between INNER JOIN and LEFT JOIN"},
            {"role": "assistant", "content": """Great question! Here's a clear comparison:

## INNER JOIN
Returns only rows where there's a match in **both** tables.

```sql
SELECT c.name, o.order_id
FROM customers c
INNER JOIN orders o ON c.id = o.customer_id;
-- Only returns customers WHO HAVE orders
```

## LEFT JOIN (or LEFT OUTER JOIN)
Returns **all rows from the left table**, plus matches from the right table (NULL if no match).

```sql
SELECT c.name, o.order_id
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id;
-- Returns ALL customers, even those without orders
-- order_id will be NULL for customers with no orders
```

## Visual Example
```
Customers: [A, B, C]     Orders: [A has 2, B has 1]

INNER JOIN result: A, A, B    (3 rows - only matching)
LEFT JOIN result:  A, A, B, C (4 rows - C has NULL order_id)
```

## When to use which:
- **INNER JOIN**: When you only want records that exist in both tables
- **LEFT JOIN**: When you want all records from the main table, regardless of matches

Need help deciding which to use for a specific query?"""}
        ]
    }
]

print(f"📊 Sample dataset with {len(sample_data)} examples")

### 2.1 Your Contribution: Define Training Examples

**Why this matters:** The quality of your fine-tuned model depends entirely on your training data. You need to provide examples that represent:
- The types of questions users will ask
- The response style and format you want
- Edge cases and tricky scenarios

**Your task:** Add at least 10 more training examples below. Consider:
- What domain expertise should the model have?
- What's the ideal response length and format?
- What common mistakes should it avoid?

In [ ]:
# ============================================================
# TODO: Add your training examples here
# ============================================================

def create_example(user_query: str, assistant_response: str) -> dict:
    """Helper to create a training example with consistent system prompt."""
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_query},
            {"role": "assistant", "content": assistant_response}
        ]
    }

# Add your examples here:
my_training_examples = [
    # Example: Uncomment and modify
    # create_example(
    #     user_query="How do I optimize a slow query?",
    #     assistant_response="Here's how to optimize..."
    # ),
]

# Combine all data
all_training_data = sample_data + my_training_examples
print(f"📊 Total training examples: {len(all_training_data)}")

if len(all_training_data) < 10:
    print("⚠️  Recommendation: Add more examples for better results (10+ minimum, 50+ recommended)")

In [ ]:
# Validate dataset format

def validate_dataset(data: list) -> bool:
    """Validate that dataset follows Together AI format requirements."""
    errors = []
    
    for i, example in enumerate(data):
        if "messages" not in example:
            errors.append(f"Example {i}: Missing 'messages' key")
            continue
            
        messages = example["messages"]
        
        if len(messages) < 2:
            errors.append(f"Example {i}: Need at least 2 messages (user + assistant)")
            
        for j, msg in enumerate(messages):
            if "role" not in msg:
                errors.append(f"Example {i}, message {j}: Missing 'role'")
            if "content" not in msg:
                errors.append(f"Example {i}, message {j}: Missing 'content'")
            if msg.get("role") not in ["system", "user", "assistant"]:
                errors.append(f"Example {i}, message {j}: Invalid role '{msg.get('role')}'")
    
    if errors:
        print("❌ Validation errors:")
        for error in errors[:10]:  # Show first 10 errors
            print(f"   - {error}")
        return False
    else:
        print("✅ Dataset validation passed!")
        return True

validate_dataset(all_training_data)

In [ ]:
# Save to JSONL file

TRAIN_FILE = "training_data.jsonl"

with open(TRAIN_FILE, 'w') as f:
    for example in all_training_data:
        f.write(json.dumps(example) + '\n')

print(f"✅ Saved {len(all_training_data)} examples to {TRAIN_FILE}")

# Show file size
import os
file_size = os.path.getsize(TRAIN_FILE)
print(f"   File size: {file_size / 1024:.1f} KB")

## 3. Upload Dataset to Together AI

In [ ]:
# Upload the training file

print("📤 Uploading dataset to Together AI...")

file_response = client.files.upload(
    file=open(TRAIN_FILE, "rb"),
    purpose="fine-tune"
)

FILE_ID = file_response.id
print(f"✅ Upload complete!")
print(f"   File ID: {FILE_ID}")
print(f"   Status: {file_response.status}")

In [ ]:
# Check file processing status

import time

def wait_for_file_processing(file_id: str, timeout: int = 300):
    """Wait for file to be processed."""
    start = time.time()
    
    while time.time() - start < timeout:
        file_info = client.files.retrieve(file_id)
        status = file_info.status
        
        print(f"   Status: {status}")
        
        if status == "processed":
            print("✅ File processed successfully!")
            return True
        elif status == "error":
            print(f"❌ Processing error: {file_info.error}")
            return False
            
        time.sleep(5)
    
    print("⏰ Timeout waiting for file processing")
    return False

wait_for_file_processing(FILE_ID)

## 4. Start Fine-Tuning Job

### Available Base Models on Together:

| Model | Size | Best For | Cost |
|-------|------|----------|------|
| `meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo` | 8B | General, fast | $$ |
| `meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo` | 70B | High quality | $$$$ |
| `mistralai/Mistral-7B-Instruct-v0.3` | 7B | Good balance | $$ |
| `Qwen/Qwen2.5-7B-Instruct-Turbo` | 7B | Multilingual | $$ |

In [ ]:
# ============================================================
# Configure Fine-Tuning Job
# ============================================================

# Your model's name (will be used for deployment)
MODEL_SUFFIX = "sql-assistant-v1"  # Change this!

# Base model to fine-tune
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

# Training hyperparameters
config = {
    "n_epochs": 3,                    # Number of training epochs
    "learning_rate": 1e-5,            # Learning rate
    "batch_size": 4,                  # Batch size (adjust based on model size)
    "warmup_ratio": 0.1,              # Warmup steps as ratio of total
    "n_checkpoints": 1,               # Save checkpoints
}

print("📋 Fine-tuning configuration:")
print(f"   Base model: {BASE_MODEL}")
print(f"   Output suffix: {MODEL_SUFFIX}")
for k, v in config.items():
    print(f"   {k}: {v}")

In [ ]:
# Create fine-tuning job

print("🚀 Starting fine-tuning job...")

job = client.fine_tuning.create(
    model=BASE_MODEL,
    training_file=FILE_ID,
    suffix=MODEL_SUFFIX,
    hyperparameters={
        "n_epochs": config["n_epochs"],
        "learning_rate": config["learning_rate"],
        "batch_size": config["batch_size"],
        "warmup_ratio": config["warmup_ratio"],
        "n_checkpoints": config["n_checkpoints"],
    }
)

JOB_ID = job.id
print(f"✅ Fine-tuning job created!")
print(f"   Job ID: {JOB_ID}")
print(f"   Status: {job.status}")

## 5. Monitor Training Progress

In [ ]:
def check_job_status(job_id: str):
    """Check fine-tuning job status."""
    job = client.fine_tuning.retrieve(job_id)
    return job

def monitor_job(job_id: str, poll_interval: int = 60):
    """Monitor job until completion."""
    print(f"📊 Monitoring job {job_id}...")
    print("   (Fine-tuning typically takes 10-60 minutes)\n")
    
    while True:
        job = check_job_status(job_id)
        status = job.status
        
        print(f"   [{time.strftime('%H:%M:%S')}] Status: {status}")
        
        if hasattr(job, 'events') and job.events:
            latest_event = job.events[-1]
            print(f"                  Event: {latest_event.message}")
        
        if status == "completed":
            print(f"\n✅ Fine-tuning completed!")
            print(f"   Model: {job.output_name}")
            return job
        elif status in ["failed", "cancelled"]:
            print(f"\n❌ Job {status}")
            if hasattr(job, 'error'):
                print(f"   Error: {job.error}")
            return job
        
        time.sleep(poll_interval)

In [ ]:
# Quick status check
job_status = check_job_status(JOB_ID)
print(f"Job ID: {JOB_ID}")
print(f"Status: {job_status.status}")
if hasattr(job_status, 'output_name') and job_status.output_name:
    print(f"Model: {job_status.output_name}")

In [ ]:
# Monitor until completion (uncomment to run)
# completed_job = monitor_job(JOB_ID)

## 6. Test Your Fine-Tuned Model

In [ ]:
# Get your fine-tuned model name
job_info = check_job_status(JOB_ID)
FINE_TUNED_MODEL = job_info.output_name if hasattr(job_info, 'output_name') else None

if FINE_TUNED_MODEL:
    print(f"🎯 Fine-tuned model: {FINE_TUNED_MODEL}")
else:
    print("⚠️ Model not ready yet - check job status above")
    # You can manually set it if you know the name:
    # FINE_TUNED_MODEL = "your-username/Meta-Llama-3.1-8B-Instruct-Turbo-sql-assistant-v1"

In [ ]:
def chat(model: str, user_message: str, system_prompt: str = None) -> str:
    """Send a message to a model and get the response."""
    messages = []
    
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    messages.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
# Test the fine-tuned model

test_queries = [
    "How do I write a query to get the average order value per month?",
    "What's the difference between WHERE and HAVING?",
    "Write a query to find users who haven't logged in for 30 days"
]

if FINE_TUNED_MODEL:
    print("🧪 Testing fine-tuned model\n")
    print("=" * 70)
    
    for query in test_queries:
        print(f"\n👤 User: {query}")
        print("-" * 50)
        
        response = chat(
            model=FINE_TUNED_MODEL,
            user_message=query,
            system_prompt=SYSTEM_PROMPT
        )
        print(f"🤖 Assistant:\n{response}")
        print("=" * 70)
else:
    print("⚠️ Please set FINE_TUNED_MODEL first")

## 7. Compare Base vs Fine-Tuned

In [ ]:
def compare_models(query: str):
    """Compare responses from base and fine-tuned models."""
    print(f"Query: {query}\n")
    print("=" * 70)
    
    print("\n📌 BASE MODEL:")
    print("-" * 50)
    base_response = chat(BASE_MODEL, query, SYSTEM_PROMPT)
    print(base_response)
    
    if FINE_TUNED_MODEL:
        print(f"\n🎯 FINE-TUNED MODEL:")
        print("-" * 50)
        ft_response = chat(FINE_TUNED_MODEL, query, SYSTEM_PROMPT)
        print(ft_response)
    
    print("\n" + "=" * 70)

# Compare
compare_models("Write a query to calculate customer lifetime value")

## 📚 Resources

- [Together AI Documentation](https://docs.together.ai/)
- [Fine-Tuning Guide](https://docs.together.ai/docs/fine-tuning)
- [Model Catalog](https://together.ai/models)
- [Pricing](https://together.ai/pricing)